In [51]:
from __future__ import print_function
from time import time

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

In [67]:
import sys
import MeCab
STOPWORDS = []
SKIP_WORD_CLASSES = ["BOS/EOS"]

class MA:
    def __init__(self):
        self.mecab = MeCab.Tagger("-Ochasen -d /usr/local/lib/mecab/dic/mecab-ipadic-neologd")
        self.stopwords = STOPWORDS
        self.skip_word_classes  = SKIP_WORD_CLASSES
        
    def parse(self, sentence):
        node = self.mecab.parseToNode(sentence)
        res = []
        while node:
            cat1 = node.feature.split(",")[0]
            if node.surface in STOPWORDS:
                node = node.next
                continue
            if cat1 in SKIP_WORD_CLASSES:
                node = node.next
                continue
            res.append(node.surface)
            node = node.next
        return " ".join(res)

import os
dirname = "/var/pti/scrape"
ma = MA()
def load_file(filename):
    with open(filename, "r") as f:
        try:
            res = []
            for line in f:
                line = line.rstrip()
                line = ma.parse(line)
                if line is not None:
                    res.append(line)
        except Exception as e:
            return None
    
    return " ".join(res)

def load_articles():
    articles = []
    idx2doc = []
    for basename in sorted(os.listdir(dirname)):
        basename = basename.rstrip()
        if not basename.endswith(".txt"):
            continue
        filepath = os.path.join(dirname, basename)
        article = load_file(filepath)
        if article is None:
            continue
        idx2doc.append(basename.rstrip(".txt"))
        articles.append(article)
    return idx2doc, articles

(idx2doc, dataset) = load_articles()

In [68]:
print(idx2doc)
print(datasets)

['1', '2']
['\u3000 神奈川県小田原市 で 生活保護受給者 の 自立支援 を 担当 する 市職員 ら が 「 不正受給 は クズ だ 」 と の 趣旨 の 英文 が 書か れ た ジャンパー を 着 て い た 問題 で 、 職員 ら が 「 生活保護 悪 撲滅 チーム 」 を 示す 「 ＳＨＡＴ 」 の マーク を 袖 に 付け た 夏 用 の ポロシャツ を 製作 し 、 業務 で 着用 し て い た こと が ３ 日 、 市 へ の 取材 で 分かっ た 。 \u3000 市 は 不適切 な マーク だ として 、 同日 以降 の 使用 を 禁止 し た 。 \u3000 市 に よる と 、 ポロシャツ は 、 ジャンパー が 作ら れ た ２ ０ ０ ７ 年 の 翌年 夏 に 製作 さ れ 、 現在 も ２ ８ 人 の 職員 が 所持 し て いる 。 青 や 紺 など 数種類 あり 、 １ 着 ２ 千 円 前後 で 各 職員 が 個人 として 購入 し て い た 。  トランプ大統領 の 円安 誘導 批判 は 実力行使 を 伴わ ぬ 空砲 で あり 、 神通力 を 失う の は 時間 の 問題 だ と 野村証券 の 池田雄之輔 氏 は 分析 。 \xa0 記事 の 全文 ドル円 相場 波乱 の 芽 は 、 中期的 に は トランプ政権 の 動向 より も 金融政策 に 潜ん で いる と バークレイズ 証券 の 門田 真一郎 氏 は 指摘 。 \xa0 記事 の 全文 トムソン・ロイター は 世界最大 級 の 国際 マルチメディア 通信社 です 。 マーケット ニュース 、 ビジネスニュース 、 テクノロジー ニュース 、 外国為替 フォーラム 、 ワールド 、 ヘッドラインニュース 、 速報 ニュース 、 コラム 、 ブログ 、 国内 株式 、 海外 株式 、 外国為替 、 投資信託 情報 が 、 ロイター . co.jp サイト 、 ビデオ 、 モバイルサイト 、 インタラクティブ ・ テレビ プラットフォーム で ご 利用 いただけ ます 。 国内 株式 関連 の 情報 は 約 20分 遅れ 、 海外 株式 関連 の 情報 は 15分 以上 の 遅れ で 表示 し て い ます 。 為替 情報 は 10分 ごと に 更新 さ 

In [69]:
n_samples = 2000
n_features = 1000
n_topics = 2
n_top_words = 20

In [71]:
# Use tf (raw term count) features for LDA.
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=1,
                                max_features=n_features)
tf = tf_vectorizer.fit_transform(dataset)

In [72]:
lda = LatentDirichletAllocation(n_topics=n_topics, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)
lda.fit(tf)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=50.0,
             max_doc_update_iter=100, max_iter=5, mean_change_tol=0.001,
             n_jobs=1, n_topics=2, perp_tol=0.1, random_state=0,
             topic_word_prior=None, total_samples=1000000.0, verbose=0)

In [73]:
# save model
import pickle
pickle.dump(lda, open("lda.pkl", "wb"))

In [74]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

Topic #0:
ロイター ます ニュース 情報 株式 職員 トムソン 日本 10分 ポロシャツ 関連 金融 会談 マティス ご覧 問題 あり 記事 いる 負担
Topic #1:
ます 共同記者会見 不適切 選挙中 株式 全文 職員 ニュース 情報 デー トランプ大統領 取材 トムソン 経費 相場 関連 作ら おり ジャンパー トランプ



In [75]:
matrix = lda.transform(tf)
print(matrix)

[[ 0.9953175   0.0046825 ]
 [ 0.98606548  0.01393452]]


In [76]:
import pickle
for i, doc in enumerate(matrix):
    result = []
    for j, x in enumerate(doc):
        result.append((j, x))
    pickle.dump(result, open("/var/pti/topic/{}.pkl".format(idx2doc[i]), "wb"))

In [77]:
for i, doc in enumerate(matrix):
    topics = {}
    print("doc")
    for v, x in enumerate(doc):
        topics[v] = x
    count = 0
    for k, v  in sorted(topics.items(), key=lambda x:x[1], reverse=True):
        print ("Topic:{}, prob:{}".format(k, v))
        count += 1
        if count >= 2:
            break

doc
Topic:0, prob:0.9953174983462851
Topic:1, prob:0.004682501653714899
doc
Topic:0, prob:0.9860654820955974
Topic:1, prob:0.013934517904402517
